<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/ADMET_M_AT_clean_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet

! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

In [3]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [10]:
from scripts.eval_utils import *
from scripts.preprocess_mols import *
import torch.nn as nn
from scripts.preprocess_mols import collect_data
from models.dataloader_AT import *

# train_datasets = get_graph_dataset(trains, met_cls)
# train_loader = get_graph_dataloader(train_datasets, loader_params)

batch_size = 64
loader_params ={'batch_size': batch_size, 'shuffle': True}
params_test = {'batch_size': batch_size, 'shuffle': False}

def get_loader(df, name, loader_params=loader_params):
    dataset = get_graph_dataset(df, name)
    loader = get_graph_dataloader(dataset, loader_params)
    return loader

def get_multi_loader(trains, valids, tests, name, loader_params=loader_params):
    train_loader = get_loader(trains, name, loader_params)
    valid_loader = get_loader(valids, name, loader_params)
    test_loader  = get_loader(tests,  name, params_test) # should not shuffle
    return train_loader, valid_loader, test_loader

import torch.nn.functional as F
def evaluate_(preds_, labels_, masks_, names, IS_R):
    if IS_R == False:
        for i, name in enumerate(names):
            # try: name = name.split('_')[0]
            # except: pass
            print('*'*15, name, '*'*15)
            probs = F.sigmoid(preds_[:, i][~masks_[:, i]])
            label = labels_[:, i][~masks_[:, i]]
            probs = probs.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            assert len(probs) == len(label)
            preds = get_preds(0.5, probs)
            evaluate(label, preds, probs)
            print()

In [ ]:
met_cls = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
            'CYP1A2_Veith', 'CYP2C9_Veith']

IS_R = False # is regression task
SCALE = False

if IS_R: loss_fn = nn.MSELoss(reduction='sum') # if regression
else: loss_fn = nn.BCEWithLogitsLoss(reduction='sum') # if classification

trains, valids, tests = collect_data(met_cls, IS_R=False)
train_loader, valid_loader, test_loader = get_multi_loader(
                                trains, valids, tests, met_cls, loader_params)

In [15]:
from dgllife.utils import EarlyStopping, Meter
import torch
import numpy as np
from models.train_AT import train_epoch

cuda = torch.cuda.is_available()
if cuda: device = 'cuda'
else: device = 'cpu'

from models.dataloader_AT import get_model_AT
model = get_model_AT(met_cls, n_layers=5, graph_feat_size=300,
                     dropout=0.3, device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)
stopper = EarlyStopping(mode='lower', patience=30)
n_epochs = 500
best_epoch = 0
min_loss = 100000

IS_R = False # is regression task

if IS_R: loss_fn = nn.MSELoss(reduction='sum') # if regression
else: loss_fn = nn.BCEWithLogitsLoss(reduction='sum') # if classification

In [16]:
if best_epoch != 0:
    model.load_state_dict(torch.load('ckpt_M.pt', map_location=device))

for epoch in range(best_epoch, best_epoch + n_epochs):
    score = train_epoch(epoch, model, train_loader, loss_fn, optimizer)
    val_score = train_epoch(epoch, model, valid_loader, loss_fn)
    early_stop = stopper.step(val_score[1], model)
    if val_score[1] < min_loss and epoch > 3:
        print(f'prev min loss {min_loss:.3f}, now loss {val_score[1]:.3f} |',
              f'save model at epoch: {epoch}')
        min_loss = val_score[1]
        torch.save(model.state_dict(), 'ckpt_M.pt')
        best_epoch = epoch

    if early_stop: print('early stop'); break

Epoch:144, [Train] R2: 0.564, Loss: 0.527
Epoch:144, [Valid] R2: 0.458, Loss: 0.427
EarlyStopping counter: 30 out of 30
early stop


In [17]:
from models.train_AT import eval_AP

model.load_state_dict(torch.load('ckpt_M.pt', map_location=device))
preds_, labels_, masks_ = eval_AP(model, False, test_loader)
evaluate_(preds_, labels_, masks_, met_cls, False)

Evaluate on test sets
using BCELOSSwithdigits


100%|██████████| 145/145 [00:18<00:00,  7.98it/s]


total_loss: 0.476
*************** CYP2C19_Veith ***************
Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.841  &  0.841  &          0.812  &     0.850  &0.833  &0.830 &0.913 &   0.681 &   0.896
ROC-AUC: 0.841
PR-AUC: 0.759

*************** CYP2D6_Veith ***************
Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.882  &  0.756  &          0.741  &     0.557  &0.956  &0.636 &0.886 &   0.575 &   0.720
ROC-AUC: 0.756
PR-AUC: 0.495

*************** CYP3A4_Veith ***************
Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.819  &  0.812  &          0.788  &     0.769  &0.854  &0.778 &0.910 &   0.626 &   0.873
ROC-AUC: 0.812
PR-AUC: 0.701

*************** CYP1A2_Veith ***************
Accuracy, weighted accuracy, precision, recall/SE, SP,     F1,     AUC,     MCC,     AP
& 0.879  &  0.879  &          0.866  &     0.879  &0.879  &0.873 &0.945 &   0.757 &

In [21]:
# from tdc.benchmark_group import admet_group
# group = admet_group(path = 'data/')
# pred_list = []
# for seed in [0, 1, 2, 3, 4, 5]:
#     pred_dict = {}
#     for i, name in enumerate(met_cls):
#         benchmark = group.get(name)
#         name_spec = benchmark['name']
#         if name.lower() == name_spec:
#             test = benchmark['test']
#             test  = rename_cols(test[['Drug', 'Y']],  name)
#             test_loader = get_loader(test, [name], params_test)
#             for idx, batch_data in enumerate(test_loader):
#                 smiles, bg, labels, masks = batch_data
#                 n_feats = bg.ndata.pop('hv')
#                 e_feats = bg.edata.pop('he')
#                 pred = model(bg, n_feats, e_feats)
#                 if idx == 0: preds_ = pred.cpu()
#                 else: preds_ = torch.cat([preds_, pred.cpu()], dim=0)
#             pred_dict[name_spec] = get_preds(0.5,
#                 F.sigmoid(preds_[:, i]).cpu().detach().numpy())
#             pred_list.append(pred_dict)

[Parallel(n_jobs=2)]: Done 1660 tasks      | elapsed:    2.5s
[Parallel(n_jobs=2)]: Done 2467 out of 2467 | elapsed:    3.6s finished


Processing dgl graphs from scratch...


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 1660 tasks      | elapsed:    2.9s
[Parallel(n_jobs=2)]: Done 2419 out of 2419 | elapsed:    3.9s finished


In [23]:
# results = group.evaluate_many(pred_list)
# results

{'cyp2d6_veith': [0.651, 0.0],
 'cyp3a4_veith': [0.8, 0.0],
 'cyp2c9_veith': [0.763, 0.0]}